In [12]:
import pandas as pd
import re
from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

In [6]:
!pip install pymorphy2

  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=ea41e6e3f4a722f745d10557ac097881759eee3fa5325b68f585a313a11c977c
  Stored in directory: /Users/michaelmiller/Library/Caches/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [4]:
jokes = pd.read_csv("/Users/michaelmiller/Documents/GitHub/He_who_laughs_last/Data_preprocessing_classification/data/jokes.csv")

In [5]:
jokes

,Unnamed: 0,0
0,0,Колобок - это сказка про взросление. Парень ух...
1,1,Нам всё-таки удалось распространить свою идеол...
2,2,Fоrbеs оценил состояние российских миллиардеро...
3,3,Есть свидетельство о рождении. Есть свидетельс...
4,4,"Пожарные оперативно потушили здание налоговой,..."
...,...,...
265744,265744,Путин подходит к унитазу открывает крышку и го...
265745,265745,Арестован фигурант дела о хищениях в системе Ж...
265746,265746,"Ремейк сказки ""Золушка"" на современном телевид..."
265747,265747,Встретились как то на дереве два дятла. Хотели...


In [8]:


patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [10]:
jokes.drop('Unnamed: 0',axis=1, inplace=True)

In [13]:
lemm = []
for i in tqdm(jokes['0']):
    lemm.append(lemmatize(i))

100%|██████████████████████████████████| 265749/265749 [07:30<00:00, 590.46it/s]


In [14]:
lemm

[['колобок',
  'это',
  'сказка',
  'взросление',
  'парень',
  'уходить',
  'родительский',
  'дом',
  'успешно',
  'преодолевать',
  'жизненный',
  'этап',
  'социализация',
  'заяц',
  'работа',
  'волк',
  'бытовой',
  'трудность',
  'медведь',
  'встречать',
  'женщина',
  'который',
  'обман',
  'завлекать',
  'уничтожать',
  'личность'],
 ['мы',
  'всё',
  'таки',
  'удаться',
  'распространить',
  'идеология',
  'менталитет',
  'запад',
  'теперь',
  'бояться',
  'квитанция',
  'жкх'],
 ['быть',
  'оценить',
  'состояние',
  'российский',
  'миллиардер',
  'предынфарктный'],
 ['есть',
  'свидетельство',
  'рождение',
  'есть',
  'свидетельство',
  'смерть',
  'где',
  'свидетельство',
  'жизнь',
  '©',
  'михаил',
  'жванецкий'],
 ['пожарный', 'оперативно', 'потушить', 'здание', 'налоговый'],
 ['как',
  'грамотно',
  'написать',
  'письмо',
  'саппорт',
  'уважаемый',
  'господин',
  'уже',
  'год',
  'являться',
  'ваш',
  'клиент',
  'испытывать',
  'чувство',
  'глубокий',
 

In [19]:
lemmatized = pd.DataFrame(lemm)

TypeError: object of type 'NoneType' has no len()

In [22]:
jokes['lemmatized'].to_csv("/Users/michaelmiller/Documents/GitHub/He_who_laughs_last/data/lemmatized_russian.csv",index=False)

In [ ]:
jokes